### Kilauea summit earthquakes - 2018 Eruption

Nicole Hoffman

Abstract:

The 2018 eruption of the lower East Rift Zone (LERZ) of Kilauea volcano in Hawaii was the largest in over 200 years. While the volcano had active vents since 1983, changes began to occur in March 2018 which indicated new patterns of magma movement and the possibility of a more significant eruption in the near future. Increasing seismicity in the LERZ combined with ground deformation was followed by LERZ eruptive fissures opening in early May. 

At the summit, magma was draining from the shallow subsurface reservoir, removing the support for the caldera rocks above. As magma continued to drain, the rate of summit earthquakes increased and an interesting pattern emerged: increasing earthquake rate (mostly < M4) followed by a larger magnitude collapse event (~M5), and then an abrupt decrease in the earthquake rate. After a few hours with very few earthquakes, the process would begin again. A total of 62 collapse events occurred between early May and August 2018, after which the eruption ceased.

The pattern of earthquakes and collapse events became remarkably consistent near the end of May. This visualization focuses on the summit earthquakes and explosive events from May 29 to August 3. Using the explosive event times starting May 29, the summit earthquakes were placed into “sets”, allowing the comparison of the properties of each set and how they change during the eruption. 

Each plot is intended to visualize a different aspect of the summit earthquake sets. The top scatter plot shows an overview of all of the earthquakes (colored dots) and explosive events (diamonds) over time. When hovering over each explosive event, the magnitude and depth is displayed.

The center left plot is a histogram of the number of events in each set. Individual or multiple sets can be selected (hold down the shift-key) and those selections are linked to the other plots, highlighting the selected set(s). To the right of the histogram is a scatter plot of the spatial distribution of the earthquakes (the circular shape results from an initial selection of earthquakes in a 6 km radius circular region centered on the summit).

The bottom left plot is the frequency-magnitude diagram (FMD) for each set (a cumulative sum of events at each magnitude; the smallest magnitude contains the total number of events). When a set(s) is selected in the histogram, the corresponding FMD(s) is displayed.
 
The bottom right plot shows the estimated b-value for the Gutenberg-Richter distribution given by: log N = a-bM, where N is the total number of earthquakes greater than magnitude M, the a-value is the total seismicity, and the b-value is the proportion of large to small earthquakes. The b-value was estimated for a range of completeness magnitudes (Mc 1.8-2.4). Using the interactive legend on the right, a completeness magnitude(s) can be selected to show how this affects the estimated b-value.

#### Downloading the data

The earthquake and explosion events were downloaded from the [USGS earthquake catalog](https://earthquake.usgs.gov/earthquakes/search/). These files are available in the repository along with this notebook.

In [1]:
# Imports for data cleaning
import pandas as pd
import numpy as np
import math

# Plotting imports for Altair
import altair as alt
from altair import Chart, Color, Scale

# Automatically load the data from disk using:
alt.data_transformers.enable('json')

# Uncomment the following line to run in a Notebook (rather than Jupyter Lab)
#alt.renderers.enable('notebook')

DataTransformerRegistry.enable('json')

In [2]:
# Load in the data, separated by month because of USGS download limits

# Keep only some of the columns
colkeep = ['time','depth','mag','magType', 'latitude', 'longitude']

# Load in the earthquakes
aprmay = pd.read_csv('/home/nicole/earthquakes/kilauea/data/aprmay_M15.csv', usecols=colkeep)
june = pd.read_csv('/home/nicole/earthquakes/kilauea/data/june_M15.csv', usecols=colkeep)
july = pd.read_csv('/home/nicole/earthquakes/kilauea/data/july_M15.csv', usecols=colkeep)
aug = pd.read_csv('/home/nicole/earthquakes/kilauea/data/aug_M15.csv', usecols=colkeep)

events = pd.concat([aprmay,june,july,aug])

# Load in the summit explosions
explode = pd.read_csv('/home/nicole/earthquakes/kilauea/data/explosions_M4.csv', usecols=colkeep)

# Convert to datetime
events['DTtime'] = pd.to_datetime(events['time'])
explode['DTtime'] = pd.to_datetime(explode['time'])

# Order by ascending date
events.sort_values(by=['DTtime'], inplace=True, ascending=True)
explode.sort_values(by=['DTtime'], inplace=True, ascending=True)

#### Group the earthquakes

During the course of the eruption, the earthquake rate increased prior to each explosive summit event. In order to compare characteristics of the earthquakes over time, the earthquakes are grouped into sets determined by the explosion times.

In [3]:
# Start with the first of the larger explosions (>M5)
# consistent explosions begin at index 12 (2018-05-29 11:56:11.570)

# Delete the last time from start_times
start_times = explode.DTtime[12:-1].tolist()
# Begin the end times one time after the start times
end_times = explode.DTtime[13:].tolist()

print(start_times[0])
print(end_times[0])

2018-05-29 11:56:11.570000
2018-05-30 20:53:50.830000


In [4]:
# Assign earthquakes to their "explosion set"
# (all events beefore May 29 are in set 0)
events['ex_set'] = 0

# The explosions begin with earthquakes in set 1
i=1 # start with group 1
for start, end in zip(start_times, end_times):
    mask = (events['DTtime'] >= start) & (events['DTtime'] < end)
    events.loc[mask,'ex_set'] = i
    i+= 1 # increase counter for the next group

# Events after the last explosion time are grouped together
mask = events['DTtime'] > end_times[-1]
events.loc[mask, 'ex_set'] = i

#### Frequency-magnitude data

Using the set numbers determined above, the earthquakes for each magnitude bin (tenths of a magnitude) are summed to make the frequency-magnitude plots.

In [5]:
# Drop the uneeded columns (time, magnitude, location aren't needed for this part)
events_mags = events.drop(['DTtime','magType', 'latitude', 'longitude'], axis=1)

# Rounds magnitudes to the nearest tenths (pseudo-binning)
events_mags['mag'] = events_mags['mag'].round(decimals=1)

# Pivot to get explode sets as columns and find earthquake counts for each magnitude
quake_sum = events_mags.pivot_table(index='mag', columns='ex_set', 
                        values='depth', aggfunc='count', fill_value=0)

quake_sum_div = quake_sum.div(quake_sum.max(axis=0), axis=1)

# Flatten to put set numbers back as a value in the rows
# Convert to dataframe & give new column a name (was a series before)
events_sum = quake_sum_div.unstack().to_frame(name='counts').reset_index()

# Reverse cumulative sum of each column (whole dateframe) and sort
quake_cumsum = quake_sum[::-1].cumsum()
quake_cumsum.sort_values('mag', inplace=True)

# Normalize the values by the total counts (max value) for each explode set
quake_cumsum_norm = quake_cumsum.div(quake_cumsum.max(axis=0), axis=1)

# Flatten as above
events_cumsum = quake_cumsum_norm.unstack().to_frame(name='cs_counts').reset_index()

# Add the cs_counts column to the events_sum dataframe
events_sum['cs_counts'] = events_cumsum['cs_counts'].values

#### Calculate the statistics for the set of earthquakes

The b-values (and associated error in the b-value estimation) are estimated for a range of completeness magnitudes.

In [6]:
# Keep only magnitude and set number
events_stats = events[['mag','ex_set']]

# Filter out set 0 (before regular pattern of explosions started)
events_stats = events_stats[events_stats.ex_set != 0]

In [7]:
# Define the class to calculate the relevant statistics for each set

class SetStats:
    """Returns statistics calculated from earthquake sets"""
    
    def __init__(self, mags, magC):
        self.mags = mags
        self.mc = magC
        self.Ntot = len(self.mags)
        self.magmu = np.mean(self.mags)

    def getB(self):
        """estimate the b-values for the list of events"""
        b_est = math.log10(math.e)/(self.magmu-(self.mc - 0.5*0.1))
        return b_est
    
    def getB_err(self):
        """uncertainity in the b-value estimate"""
        b_err = self.getB()/math.sqrt(self.Ntot)
        return b_err
    
    def getA(self):
        """estimate a using the b-value for the set"""
        a = math.log10(self.Ntot) - self.getB()*self.mc
        return a

In [8]:
# Find the b values for a range of min. magnitudes

# minimum magnitudes of completeness
mag_min = [1.8,1.9,2.0,2.1,2.2,2.3,2.4]

bvals_df = pd.DataFrame(index=events_stats.ex_set.unique(), columns=mag_min)
berrs_df = pd.DataFrame(index=events_stats.ex_set.unique(), columns=mag_min)

for setnum in events_stats.ex_set.unique():

    for magc in mag_min:
        
        events_list = events_stats.loc[(events_stats.mag >= magc) & (events_stats.ex_set == setnum)].mag.values
        statsobj = SetStats(events_list, magc)
        b = statsobj.getB()
        bvals_df.loc[setnum, magc] = b
        berr = statsobj.getB_err()
        berrs_df.loc[setnum, magc] = berr 

bvals_df['ex_set'] = range(1,51)
berrs_df['ex_set'] = range(1,51)

In [9]:
# convert both dataframes to long-form
bvals_long = pd.melt(bvals_df, id_vars=['ex_set'], var_name='mag_c', value_name='bvals')
berrs_long = pd.melt(berrs_df, id_vars=['ex_set'], var_name='mag_c', value_name='berrs')

# add the berrs column to bvals
bvals_long['berrs'] = berrs_long.berrs

# round the bvals and berrs to 0.01
cols=['bvals','berrs']
from decimal import Decimal
bvals_long[cols] = bvals_long[cols].astype(float).round(2)

In [10]:
# Begin the events and explosion times as the start of the repeating events
# (don't need to plot the earlier events in the following plot)
events = events[events.ex_set != 0]
events_sum = events_sum[events_sum.ex_set != 0]
explode = explode[explode.DTtime >= '2018-05-29 11:56:11.570000']

# Add the mag_c column to the events_sum df
events_sum['mag_c'] = events_sum.mag

In [11]:
# Select individual or multiple bars
click_multi = alt.selection(type='multi', encodings=['x'])

# Scatter plot of the earthquakes
quakes = alt.Chart(events, width=1200, height=200).mark_circle(
    opacity=0.75, size=3).encode(
    x=alt.X(
        'DTtime:T', 
        axis=alt.Axis(title='Date (2018)')
    ),
    y=alt.Y(
        'mag:Q', 
        scale=alt.Scale(domain=(1.5, 5.6)),
        axis=alt.Axis(title='earthquake magnitude')
    ),
    color=alt.Color('ex_set:Q', legend=None)
    ).transform_filter(
    click_multi    
    )

# Add the explosive events to the earthquake scatter plot
explode = alt.Chart(explode).mark_point(
    opacity=0.5, size=40, shape='diamond', color='black').encode(
    x='DTtime:T',
    y=alt.Y(
        'mag:Q', 
        scale=alt.Scale(domain=(1.5, 5.6))
    ),
    tooltip=['mag', 'depth'],
    ).transform_filter(
    click_multi
    ) 

# Scatter plot of the summit earthquakes in map view
# latitude/longitude ratio: width = 0.94 * height
h_latlon = 225
w_latlon = 0.94 * h_latlon

latlon = alt.Chart(events, width=w_latlon, height=h_latlon).mark_circle(size=10).encode(
    x=alt.X('longitude:Q',
            scale=alt.Scale(domain=(-155.3459, -155.2209))
           ),
    y=alt.Y('latitude:Q',
            scale=alt.Scale(domain=(19.3444, 19.4694))
           ),
    color=alt.condition(
        click_multi, 'ex_set', alt.value('lightgray'), legend=None),
    opacity=alt.value(0.5)
    ).transform_filter(
    click_multi    
    )

# Histogram of the total events for each set
bars = alt.Chart(events, width=800, height=225).mark_bar(size=12, clip=True).encode(
    x=alt.X('ex_set',
            axis=alt.Axis(title='explosion set'),
            scale=alt.Scale(domain=[1, 52])
           ),
    y=alt.Y('count()',
           axis=alt.Axis(title='number of events')
           ),
    tooltip=['ex_set','count()'],
    color=alt.condition(
        click_multi, 'ex_set', alt.value('lightgrey'), 
        scale=alt.Scale(scheme='inferno'), legend=None
    )
    ).properties(
    selection=click_multi    
    )

# Frequency-magnitude plots for each set
fmd = alt.Chart(events_sum, width=550, height=200).mark_point(size=14).encode(
    x=alt.X('mag_c:Q',
            scale=alt.Scale(domain=(1.5, 5.0)),
            axis=alt.Axis(title='magnitude')
           ),
    y=alt.Y('cs_counts:Q',
            axis=alt.Axis(title='cumulative events')
           ),
    tooltip=['ex_set'],
    color=alt.condition(
        click_multi, 'ex_set', alt.value('lightgray'), legend=None)
    ).transform_filter(
        click_multi    
    )

# Plots for the b-values
selection = alt.selection_multi(fields=['mag_c'])
color_scatter = alt.condition(selection,
                      alt.Color('ex_set:Q', legend=None),
                      alt.value('lightgray'))

color_legend = alt.condition(selection,
                     alt.Color('mag_c:Q', legend=None),
                      alt.value('lightgray'))

scatter = alt.Chart(bvals_long, width=500, height=200).mark_point(clip=True).encode(
    x=alt.X(
        'ex_set:Q',
        scale=alt.Scale(domain=(0, 50)),
        axis=alt.Axis(title='explosion set')
    ),
    y=alt.Y(
        'bvals:Q', 
        scale=alt.Scale(domain=(0.5, 2.2)),
        axis=alt.Axis(title='estimated b value')
    ),
    tooltip=['bvals', 'berrs', 'mag_c'],
    color=color_scatter
    )

legend = alt.Chart(bvals_long, height=200).mark_circle(size=75).encode(
    y=alt.Y('mag_c:Q', axis=alt.Axis(title='completeness magnitude', orient='right'),
            scale=alt.Scale(domain=(1.8, 2.4))
           ),
    color=color_legend
    ).add_selection(
    selection
    )

# Add the legend to the b-value scatter plot
bvals_legend = scatter | legend

# Combine all plots and display
top = quakes+explode
bars_map = alt.hconcat(bars,latlon)
fmd_bvals = alt.hconcat(fmd,bvals_legend)
mychart = alt.vconcat(top, bars_map, fmd_bvals).configure(background='#E8E8E8').configure_axis(labelFontSize=16,titleFontSize=16)
mychart

mychart.save('/home/nicole/kilauea_altair_final.html')